In [1]:
import threading
import random
import time
import pandas as pd

# Функція для оцінки числа Пі методом Монте-Карло
def monte_carlo_pi(n_points, results, idx):
    inside_circle = 0
    for _ in range(n_points):
        x, y = random.random(), random.random()
        if x*x + y*y <= 1:
            inside_circle += 1
    results[idx] = inside_circle

# Запуск обчислення у головному потоці
def single_thread_pi(n_points=1_000_000):
    start = time.time()
    results = [0]
    monte_carlo_pi(n_points, results, 0)
    pi_estimate = 4 * results[0] / n_points
    elapsed = time.time() - start
    return pi_estimate, elapsed

# Запуск обчислення у кілька потоків
def multi_thread_pi(n_points=1_000_000, n_threads=4):
    points_per_thread = n_points // n_threads
    threads = []
    results = [0] * n_threads

    start = time.time()
    for i in range(n_threads):
        t = threading.Thread(target=monte_carlo_pi, args=(points_per_thread, results, i))
        threads.append(t)
        t.start()

    for t in threads:
        t.join()

    total_inside = sum(results)
    pi_estimate = 4 * total_inside / n_points
    elapsed = time.time() - start
    return pi_estimate, elapsed

# Експерименти
n_points = 1_000_000
thread_counts = [1, 2, 4, 8, 16, 32, 64]
results = []

for n_threads in thread_counts:
    if n_threads == 1:
        pi, elapsed = single_thread_pi(n_points)
    else:
        pi, elapsed = multi_thread_pi(n_points, n_threads)
    results.append({"Threads": n_threads, "Pi Estimate": pi, "Time (s)": elapsed})

df = pd.DataFrame(results)
df


,Threads,Pi Estimate,Time (s)
0,1,3.141840,0.122819
1,2,3.141944,0.185049
2,4,3.142280,0.142633
3,8,3.139480,0.157146
4,16,3.143988,0.163162
5,32,3.142384,0.183512
6,64,3.142812,0.191242
